In [1]:
import pipeline
import pandas as pd
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
data_cru = pd.read_parquet('heart_2022_no_nans.parquet')

data = pipeline.pipelines(data_cru)
X = data.create()


reports = {}

def add_model_report(reports, model_name, y_true, y_pred):
    """ Adiciona o relatório de classificação ao dicionário de relatórios. """
    reports[model_name] = classification_report(y_true, y_pred, output_dict=True)

data_cru = data_cru.drop(columns=['State'])

dados_sem_estados = X.drop(columns=['State'])

#quando for treinar os clusters é necessario que os dados estejam no formato de matriz. Ou seja, nao pode ser um dataframe

dados_em_matriz=dados_sem_estados.values

x = dados_sem_estados.drop('HadHeartAttack', axis=1)
y = dados_sem_estados['HadHeartAttack']

rus = RandomUnderSampler(random_state=42)

dados_sem_estados

,GeneralHealth,PhysicalHealthDays,MentalHealthDays,PhysicalActivities,SleepHours,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,...,AgeCategory_Age 75 to 79,AgeCategory_Age 80 or older,"TetanusLast10Tdap_No, did not receive any tetanus shot in the past 10 years","TetanusLast10Tdap_Yes, received Tdap","TetanusLast10Tdap_Yes, received tetanus shot but not sure what type","TetanusLast10Tdap_Yes, received tetanus shot, but not Tdap",HadDiabetes_No,"HadDiabetes_No, pre-diabetes or borderline diabetes",HadDiabetes_Yes,"HadDiabetes_Yes, but only during pregnancy (female)"
0,0.75,0.133333,0.000000,1.0,0.347826,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.75,0.000000,0.000000,1.0,0.217391,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.75,0.000000,0.000000,0.0,0.304348,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.25,0.166667,0.000000,1.0,0.347826,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.50,0.100000,0.500000,1.0,0.173913,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246017,0.75,0.000000,0.000000,1.0,0.217391,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
246018,0.25,0.000000,0.233333,1.0,0.260870,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
246019,0.50,0.000000,0.500000,1.0,0.260870,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
246020,1.00,0.066667,0.066667,1.0,0.260870,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest , f_classif

def metricas(x,y):
    accuracy = accuracy_score(x, y)
    print(f'Acurácia: {accuracy:.2f}')

    # 2. Precisão
    precision = precision_score(x, y)
    print(f'Precisão: {precision:.2f}')

    # 3. Revocação
    recall = recall_score(x, y)
    print(f'Revocação: {recall:.2f}')

    # 4. F1 Score
    f1 = f1_score(x, y)
    print(f'F1 Score: {f1:.2f}')

    # Exibindo a Matriz de Confusão para referência
    cm = confusion_matrix(x, y)
    print('Matriz de Confusão:')
    print(cm)
    return [accuracy,precision,recall,f1]

x = dados_sem_estados.drop('HadHeartAttack', axis=1)
y = dados_sem_estados['HadHeartAttack']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

x2, y2 = rus.fit_resample(X_train, y_train)


smote = SMOTE(random_state=42)
x3, y3 = smote.fit_resample(X_train, y_train)



selector = SelectKBest(score_func=f_classif, k=10)
X_train_best = selector.fit_transform(x2, y2)
selected_features = x2.columns[selector.get_support()]

xbest_train = x2[selected_features]
xbest_test = X_test[selected_features]

Random Forest - Modelo Normalizado

In [3]:

model = RandomForestClassifier(n_estimators=100, random_state=42)

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

y_pred_train = model.predict(X_train)

modelo_base = add_model_report(reports,'dados base',y_test, y_pred)
print('-----------')
metricas(y_train,y_pred_train)

-----------
Acurácia: 1.00
Precisão: 1.00
Revocação: 1.00
F1 Score: 1.00
Matriz de Confusão:
[[162778      0]
 [     6   9431]]


[0.9999651598292831, 1.0, 0.9993642047260782, 0.9996820012719949]

Random Forest - Modelo balanceado para baixo

In [4]:

model = RandomForestClassifier(n_estimators=100, random_state=42)

# Treinar o modelo
model.fit(x2, y2)

# Fazer previsões
y_pred = model.predict(X_test)

y_pred_train = model.predict(x2)

modelo_balanceado_baixo = add_model_report(reports,'balanceamento para baixo',y_test, y_pred)
print('-----------')
metricas(y2,y_pred_train)



-----------
Acurácia: 1.00
Precisão: 1.00
Revocação: 1.00
F1 Score: 1.00
Matriz de Confusão:
[[9437    0]
 [   0 9437]]


[1.0, 1.0, 1.0, 1.0]

Random Forest - Modelo balanceado para cima

In [5]:

bestrf = RandomForestClassifier(n_estimators=100, random_state=42)

# Treinar o modelo
bestrf.fit(x3, y3)

# Fazer previsões
y_pred = bestrf.predict(X_test)

y_pred_train = bestrf.predict(x3)

modelo_balanceado_cima = add_model_report(reports,'balanceamento para cima' ,y_test, y_pred)
print('-----------')
metricas(y3,y_pred_train)
 

-----------
Acurácia: 1.00
Precisão: 1.00
Revocação: 1.00
F1 Score: 1.00
Matriz de Confusão:
[[162778      0]
 [     1 162777]]


[0.9999969283318385, 1.0, 0.9999938566636769, 0.9999969283224033]

In [ ]:
bestrf = RandomForestClassifier(n_estimators=100, random_state=42)

# Treinar o modelo
bestrf.fit(xbest_train, y2)

# Fazer previsões
y_pred = bestrf.predict(xbest_test)

y_pred_train = bestrf.predict(xbest_train)

dados_kbest = add_model_report(reports, '10 melhores features',y_test,y_pred)
print('-----------')
metricas(y2,y_pred_train)

Plot da Tabela com os resultados

In [1]:
import pandas as pd
import plotly.graph_objects as go

def generate_comparison_table(reports):
    """ Gera uma tabela comparativa dos relatórios de classificação dos modelos. """
    data = []

    for model_name, report in reports.items():
        # Adiciona a acurácia
        accuracy = round(report.get('accuracy', 0), 2)
        data.append({'Model': model_name,
                     'Class': 'Accuracy',
                     'Precision': accuracy,
                     'Recall': accuracy,
                     'F1-Score': accuracy,
                     'Support': ''})

        # Adiciona uma linha para cada classe
        for cls, metrics in report.items():
            if cls == 'accuracy':
                continue  # Ignora a linha de 'accuracy'
            row = {'Model': model_name,
                   'Class': cls,
                   'Precision': round(metrics.get('precision', 0), 2),
                   'Recall': round(metrics.get('recall', 0), 2),
                   'F1-Score': round(metrics.get('f1-score', 0), 2),
                   'Support': round(metrics.get('support', 0), 2)}
            data.append(row)

    df_all = pd.DataFrame(data)
    return df_all

# Gerar a tabela comparativa
df_comparison = generate_comparison_table(reports)

# Exibir a tabela com Plotly
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_comparison.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_comparison[col] for col in df_comparison.columns],
               fill_color='lavender',
               align='left'))
])

fig.update_layout(title="Comparação dos Relatórios de diferentes tipos de dados no random forest")
fig.show()


NameError: name 'reports' is not defined